# Driver State Anomaly Detection

[https://dagshub.com/matejfric/driver-state](https://dagshub.com/matejfric/driver-state)

TODO: setup a pre-commit hook for https://github.com/mwouts/jupytext and Ruff

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import os
import re
from collections.abc import Mapping
from pathlib import Path
from pprint import pprint
from typing import Literal

import albumentations as albu
import dagshub
import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import numpy as np
import pytorch_lightning as L
import torch
from pytorch_lightning.callbacks import TQDMProgressBar

# Pytorch Lightning EarlyStopping callback does not recover the best weights as in Keras!
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# https://github.com/Lightning-AI/pytorch-lightning/discussions/10399,
# https://pytorch-lightning.readthedocs.io/en/1.5.10/extensions/generated/pytorch_lightning.callbacks.ModelCheckpoint.html
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger

from model import (
    AutoencoderModel,
    DatasetPathsLoader,
    DatasetSplit,
)
from model.ae import ConvDecoder, ConvEncoder, FCDecoder, FCEncoder  # noqa F401
from model.augmentation import (
    compose_transforms,
    hard_transforms,
    post_transforms,
    pre_transforms,
)
from model.common import BatchSizeDict
from model.git import get_commit_id, get_current_branch
from model.plot import (
    plot_autoencoder_reconstruction,
    plot_learning_curves,
    show_examples,  # noqa F401 TODO
    show_random,  # noqa F401 TODO
)

## Configuration

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14})

# Experiment logging
REPO_NAME = 'driver-state'
USER_NAME = 'matejfric'
dagshub.init(REPO_NAME, USER_NAME, mlflow=True)  # type: ignore

# Reproducibility
# https://lightning.ai/docs/pytorch/stable/common/trainer.html#reproducibility
SEED = 42
L.seed_everything(SEED, workers=True)

print(
    f'torch: {torch.__version__}, cuda: {torch.cuda.is_available()}, lightning: {L.__version__}'  # type: ignore
)

In [ ]:
# HYPERPARAMETERS
# ----------------------------------------
MAX_EPOCHS = 30
MONITOR = 'valid_loss'
PATIENCE = 5
IMAGE_SIZE = 256  # 224
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
LOSS_FUNCTION = 'mse'  # 'mae'
AUGMENTATION = True
TRAIN_NOISE_STD_INPUT = 0.0  # 0.025
TRAIN_NOISE_STD_LATENT = 0.0  # 0.025
HIDDEN_LAYER_SIZE = 256

# LOGGING
# ----------------------------------------
LOG_DIR = Path('logs')
EXPERIMENT_NAME = (
    f'{datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S")}-anomaly-detection-fc-ae'
)
VERSION = 0
EXPERIMENT_DIR = LOG_DIR / EXPERIMENT_NAME / f'version_{VERSION}'
DATASET_NAME = '2024-09-15-driver-segmentation-dataset'

MLFLOW_ARTIFACT_DIR = 'outputs'
METRICS_CSV_NAME = 'metrics.csv'
LEARNING_CURVES_PDF_NAME = 'learning_curves.pdf'
PREDICTIONS_PNG_NAME = 'predictions.png'
TRAIN_TRANSFORMS_JSON_NAME = 'train_transforms.json'
NOTEBOOK_NAME = 'anomaly_detection.ipynb'

# DATASET
# ----------------------------------------
DATASET_DIR = Path(f'/home/lanter/source/driver-dataset/{DATASET_NAME}')

TRAIN_SET_DIR = 'train'
VALID_SET_DIR = 'validation'
TEST_SET_DIR = 'test'

IMAGES_DIR = 'images'
MASKS_DIR = 'masks'
DEPTH_DIR = 'depth'

# NOTE: We filter out the normal images for training and validation,
# and the anomaly images for testing.

TRAIN_IMAGES = sorted((DATASET_DIR / TRAIN_SET_DIR / DEPTH_DIR).glob('*normal*.png'))
TRAIN_MASKS = sorted((DATASET_DIR / TRAIN_SET_DIR / MASKS_DIR).glob('*normal*.png'))

VALID_IMAGES = sorted((DATASET_DIR / VALID_SET_DIR / DEPTH_DIR).glob('*normal*.png'))
VALID_MASKS = sorted((DATASET_DIR / VALID_SET_DIR / MASKS_DIR).glob('*normal*.png'))

TEST_IMAGES = sorted(
    list((DATASET_DIR / TRAIN_SET_DIR / DEPTH_DIR).glob('*anomal*.png'))
    + list((DATASET_DIR / VALID_SET_DIR / DEPTH_DIR).glob('*anomal*.png'))
    + list((DATASET_DIR / TEST_SET_DIR / DEPTH_DIR).glob('*anomal*.png'))
)
TEST_MASKS = sorted(
    list((DATASET_DIR / TRAIN_SET_DIR / MASKS_DIR).glob('*anomal*.png'))
    + list((DATASET_DIR / VALID_SET_DIR / MASKS_DIR).glob('*anomal*.png'))
    + list((DATASET_DIR / TEST_SET_DIR / MASKS_DIR).glob('*anomal*.png'))
)

print(f"""Train: {(n_train_images:=len(TRAIN_IMAGES)):,}
Validation: {(n_valid_images:=len(VALID_IMAGES)):,}
Test: {(n_test_images:=len(TEST_IMAGES)):,}""")

In [ ]:
encoder = ConvEncoder()
decoder = ConvDecoder()

# encoder = FCEncoder(IMAGE_SIZE, HIDDEN_LAYER_SIZE)
# decoder = FCDecoder(IMAGE_SIZE, HIDDEN_LAYER_SIZE)

print(encoder)
print(decoder)

In [ ]:
# Simple test case for forward pass
# ---------------------------------

# Test input tensor of size (batch_size, channels, height, width)
x = torch.randn(BATCH_SIZE, 1, IMAGE_SIZE, IMAGE_SIZE)

# Forward pass through the encoder and decoder
encoded = encoder(x)
decoded = decoder(encoded)

# Check the shapes
print(f'Input shape: {x.shape}')
print(f'Encoded shape: {encoded.shape}')
print(f'Decoded shape: {decoded.shape}')

assert x.shape == decoded.shape, 'Input and output shapes do not match!'

In [7]:
# Exclude images from the training set
# TRAIN_IMAGES = [img for img in TRAIN_IMAGES if 'stribny' not in img.stem]
# TRAIN_MASKS = [mask for mask in TRAIN_MASKS if 'stribny' not in mask.stem]

In [8]:
# show_random(TRAIN_IMAGES, TRAIN_MASKS)

In [9]:
# show_random(VALID_IMAGES, VALID_MASKS)

In [10]:
# show_random(TEST_IMAGES, TEST_MASKS)

In [ ]:
xxx = np.zeros((32, 32))
xxx = np.expand_dims(xxx, axis=2)
xxx.shape

## Augmentations and Transforms

In [12]:
default_transforms = compose_transforms(
    [
        pre_transforms(image_size=IMAGE_SIZE),
        post_transforms(),
    ]
)
if AUGMENTATION:
    ae_input_transforms = compose_transforms(
        [
            pre_transforms(image_size=IMAGE_SIZE),
            hard_transforms(),
            post_transforms(),
        ]
    )
else:
    ae_input_transforms = None

## Loaders

In [13]:
dataset_loader = DatasetPathsLoader(
    train=DatasetSplit(images=TRAIN_IMAGES, masks=TRAIN_MASKS),
    valid=DatasetSplit(images=VALID_IMAGES, masks=VALID_MASKS),
    test=DatasetSplit(images=TEST_IMAGES, masks=TEST_MASKS),
)

In [14]:
batch_size_dict = BatchSizeDict(
    {'train': BATCH_SIZE, 'valid': BATCH_SIZE, 'test': BATCH_SIZE}
)
loaders = dataset_loader.get_loaders(
    # set to zero if RuntimeError: Trying to resize storage that is not resizable
    dataset='anomaly',
    num_workers=int(os.cpu_count()),  # type: ignore
    batch_size_dict=batch_size_dict,
    train_transforms=default_transforms,
    valid_transforms=default_transforms,
    test_transforms=default_transforms,
    ae_input_transforms=ae_input_transforms,
)

train_dataloader = loaders['train']
valid_dataloader = loaders['valid']
test_dataloader = loaders['test']

## Training

In [15]:
model = AutoencoderModel(
    encoder=encoder,
    decoder=decoder,
    batch_size_dict=batch_size_dict,
    learning_rate=LEARNING_RATE,
    loss_function=LOSS_FUNCTION,
    train_noise_std_input=TRAIN_NOISE_STD_INPUT,
    train_noise_std_latent=TRAIN_NOISE_STD_LATENT,
)

In [16]:
csv_logger = CSVLogger(LOG_DIR, name=EXPERIMENT_NAME, version=VERSION)
early_stopping = EarlyStopping(
    monitor=MONITOR,
    mode='min',
    patience=PATIENCE,
)
model_checkpoint = ModelCheckpoint(
    dirpath=EXPERIMENT_DIR,
    filename='{epoch}-{val_loss:3f}',
    monitor=MONITOR,
    save_top_k=1,  # save only the best model
    mode='min',
)
progress_bar = TQDMProgressBar(refresh_rate=10)

In [ ]:
trainer = L.Trainer(
    accelerator='gpu',
    logger=csv_logger,
    callbacks=[model_checkpoint, early_stopping, progress_bar],
    max_epochs=MAX_EPOCHS,
    log_every_n_steps=1,  # log every batch
    # https://lightning.ai/docs/pytorch/stable/common/trainer.html#reproducibility
    deterministic=True,
)

In [18]:
# https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
torch.set_float32_matmul_precision('medium')

In [ ]:
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=valid_dataloader,
)

## Validation

In [20]:
if False:
    # Load from MLflow
    model_name = 'pytorch-2024-10-14-220152-anomaly-detection-conv-ae'
    model_version = 1
    model_uri = f'models:/{model_name}/{model_version}'
    model_ = mlflow.pytorch.load_model(model_uri)
else:
    model_checkpoint_path = list(EXPERIMENT_DIR.glob('*.ckpt'))[0]
    model_ = AutoencoderModel.load_from_checkpoint(
        model_checkpoint_path, encoder=encoder, decoder=decoder
    )

In [ ]:
trainer_ = L.Trainer(logger=False)  # no need to log anything for validation and testing

In [ ]:
valid_metrics = trainer_.validate(model_, dataloaders=valid_dataloader, verbose=False)[
    0
]
pprint(valid_metrics)

In [ ]:
test_metrics = trainer_.test(model_, dataloaders=test_dataloader, verbose=False)[0]
pprint(test_metrics)

In [ ]:
plot_autoencoder_reconstruction(
    model_,
    test_dataloader,
    dataset_path=DATASET_DIR,
    save_path=EXPERIMENT_DIR / PREDICTIONS_PNG_NAME,
    limit=15,
    random_shuffle=False,
)

In [ ]:
plot_learning_curves(
    EXPERIMENT_DIR / METRICS_CSV_NAME,
    save_path=EXPERIMENT_DIR / LEARNING_CURVES_PDF_NAME,
    metrics={
        'mse': 'Mean Squared Error',
        'fro': 'Frobenius Norm',
        'mae': 'Mean Absolute Error',
    },
    figsize=(22, 5),
)

## Logging

In [26]:
# Save the transforms for experiment logging
if AUGMENTATION:
    albu.save(ae_input_transforms, EXPERIMENT_DIR / TRAIN_TRANSFORMS_JSON_NAME)  # type: ignore

In [27]:
def get_early_stopping_epoch() -> int | None:
    checkpoint = list(EXPERIMENT_DIR.glob('*.ckpt'))[0].stem
    pattern = r'epoch=(\d+)'
    match = re.search(pattern, checkpoint)
    if match:
        return int(match.group(1))
    else:
        return None

In [28]:
def log_dict_to_mlflow(
    dictionary: Mapping[str, int | float], type: Literal['metric', 'param']
) -> None:
    for k, v in dictionary.items():
        if type == 'metric':
            mlflow.log_metric(k, v)
        elif type == 'param':
            mlflow.log_param(k, v)

In [29]:
def get_submodule_param_count(model: L.LightningModule) -> dict[str, int]:
    """Get the number of parameters for each submodule in the model."""
    param_counts = {}
    for name, submodule in model.named_children():
        num_params = sum(p.numel() for p in submodule.parameters())
        if num_params > 0:
            param_counts[f'{name}_parameters'] = num_params
    param_counts['total_parameters'] = sum([x for x in param_counts.values()])
    return param_counts

In [ ]:
with mlflow.start_run(run_name=f'{EXPERIMENT_NAME}') as run:
    try:
        mlflow.set_tag('Branch', get_current_branch())
        mlflow.set_tag('Commit ID', get_commit_id())
        mlflow.set_tag('Dataset', DATASET_NAME)
    except Exception as e:
        print(e)

    log_dict_to_mlflow(valid_metrics, 'metric')
    log_dict_to_mlflow(test_metrics, 'metric')
    log_dict_to_mlflow(get_submodule_param_count(model_), 'param')

    mlflow.log_param('encoder', str(encoder))
    mlflow.log_param('decoder', str(decoder))
    mlflow.log_param('batch_size', BATCH_SIZE)
    mlflow.log_param('max_epochs', MAX_EPOCHS)
    mlflow.log_param('early_stopping', get_early_stopping_epoch())
    mlflow.log_param('monitor', MONITOR)
    mlflow.log_param('patience', PATIENCE)
    mlflow.log_param('image_size', IMAGE_SIZE)
    mlflow.log_param('learning_rate', LEARNING_RATE)
    mlflow.log_param('loss_function', LOSS_FUNCTION)
    mlflow.log_param('augmentation', AUGMENTATION)
    mlflow.log_param('train_noise_std_input', TRAIN_NOISE_STD_INPUT)
    mlflow.log_param('train_noise_std_latent', TRAIN_NOISE_STD_LATENT)
    mlflow.log_param('hidden_layer_size', HIDDEN_LAYER_SIZE)
    mlflow.log_param('seed', SEED)
    mlflow.log_param('n_train_images', n_train_images)
    mlflow.log_param('n_valid_images', n_valid_images)
    mlflow.log_param('n_test_images', n_test_images)

    mlflow.log_artifact(str(EXPERIMENT_DIR / METRICS_CSV_NAME), MLFLOW_ARTIFACT_DIR)
    mlflow.log_artifact(
        str(EXPERIMENT_DIR / LEARNING_CURVES_PDF_NAME), MLFLOW_ARTIFACT_DIR
    )
    mlflow.log_artifact(str(EXPERIMENT_DIR / PREDICTIONS_PNG_NAME), MLFLOW_ARTIFACT_DIR)
    if AUGMENTATION:
        mlflow.log_artifact(
            str(EXPERIMENT_DIR / TRAIN_TRANSFORMS_JSON_NAME), MLFLOW_ARTIFACT_DIR
        )
    mlflow.log_artifact(NOTEBOOK_NAME, MLFLOW_ARTIFACT_DIR)

    input = np.random.random((BATCH_SIZE, 1, IMAGE_SIZE, IMAGE_SIZE))
    signature = mlflow.models.infer_signature(input, input, dict(training=False))  # type: ignore

    # Models are versioned by default
    mlflow.pytorch.log_model(
        pytorch_model=model_,
        artifact_path='model',
        registered_model_name=f'pytorch-{EXPERIMENT_NAME}',
        signature=signature,
    )